#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час.   
Требуется построить модель для такого предсказания.

Для этого нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.



In [1]:
!pip install lightgbm
!pip install catboost

In [2]:
import numpy as np
import pandas as pd
import urllib.request
import functools as fnc
import operator as opr

from sklearn.model_selection import train_test_split,TimeSeriesSplit,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error,r2_score

from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt

In [3]:
np.random.seed(499)
get_yandex = lambda fn: urllib.request.urlretrieve(f"https://code.s3.yandex.net/datasets/{fn}", f"datasets/{fn}" )

multiply = lambda array: fnc.reduce( opr.mul,array,1 )
rmse = lambda y,pr: mean_squared_error(y,pr,squared=False) 
rmse.__name__='RMSE'
def print_metrics( y,pr, metrics = [rmse,r2_score]):
    scores = {}
    outline = ""
    for metric in metrics:
        scores[metric.__name__]= metric(y,pr)
        outline += f"{metric.__name__}: {scores[metric.__name__]}\t" 
    print(outline)   

## Подготовка

In [ ]:
try:
    get_yandex("taxi.csv")
    df = pd.read_csv('datasets/taxi.csv',parse_dates=[0],index_col=[0] )
except Exception as e:
    print(e)
    exit(1)


In [ ]:
df.sort_index(inplace=True)

_Загрузить данные и выполнить их ресемплирование по одному часу._

In [ ]:
df = df.resample('1H').sum()

In [ ]:
df.info()
df.head(),df.tail()

## Анализ

#### Общий анализ 

In [ ]:
dw = df.resample('1W').sum()
dw.index = dw.index.strftime('%d.%m')
dw.plot.bar(figsize=(20,8),title='Количество заказов по неделям');

Слабо повышающийся тренд с ускорением в конце срока

In [ ]:
dd = df.resample('1D').sum()
dd.plot(figsize=(20,8),title ="Количество заказов по дням",xlabel='Дата');

Видны  серьезные колебания со дня на день

In [ ]:
ddc = seasonal_decompose(dd)
ddc.trend.plot(figsize=(20,8),title='Тренд  по дням  ');

Тренд колчества заказов по дням  примерно соответствует недельному графику, видны слабые нерегулярные провалы . 

In [ ]:
ddc.seasonal.plot(figsize=(20,8),lw=2);
ddc.resid.plot(figsize=(20,8),ls=':',title='Сезонная компонента и остаток за март- август ');
plt.legend();

Декомпозиция показывает по 2 пика в неделю со значительными колебаниями остатка, который может превышать ритмичные колебания.

####  Месячный анализ по дням

Более подробно посмотрим июль - предпоследний месяц,   
также это последний месяц, которй будет использован в пресказаниях и последний перед ускорением роста

In [ ]:
dm = seasonal_decompose( df['2018-07-02':'2018-07-30'].resample('1D').sum() )
dm_sr= pd.DataFrame( dm.seasonal).join( dm.resid )
dm_sr.index = dm_sr.index.strftime('%d %a')
dm_sr.seasonal.plot.bar(figsize=(20,8),title='Сезонная компонета  по дням за июль ');

В сезонной компоненте выделяются пики в пятницу и меньшие - в понедельник (какие-то "вахтовые" поездки? ) а также провалы в воскресенье и вторник

In [ ]:
dm_sr.plot.bar(color=['m','y'],figsize=(20,8),title='Сезонная компонета и остатки по дням за июль ');


Остаток с сильными колебаниями, в частности можно отметить разнонаправленные колебания по вторникам и четвергам

In [ ]:
dm.trend.plot(figsize=(20,8),title='Тренд  по дням за июль ');

Тренд показывает мнотонный рост , примерно на 20% за месяц

#### Месячный анализ по дням за август

Изменилось ли что-то в данных за вагуст, которые надо предсказывать ?

In [ ]:
dm_a = seasonal_decompose( df['2018-07-30':'2018-08-26'].resample('1D').sum() )
dm_sr_a= pd.DataFrame( dm_a.seasonal).join( dm_a.resid )
dm_sr_a.index = dm_sr_a.index.strftime('%d %a')
dm_sr_a.seasonal.plot.bar(figsize=(20,8),title='Сезонная компонета  по дням за июль ');

Seasonal внешне изменился мало, но трафик по понедельникам вырос - и теперь он больше чем в пятницу.  
По другим дням отклонение от среднего  уровня не исзменилось, так что это именно увеличение числа заказов в понедельник. 

In [ ]:
dm_sr_a.plot.bar(color=['m','y'],figsize=(20,8),title='Сезонная компонета и остатки по дням за август ');

Остатки меньше, чем в июле, график более стационарный

In [ ]:
dm_a.trend.plot(figsize=(20,8),title='Тренд  по дням за август ');

Основное изменение в августовском трафике - появление нового источника заказов по понедельникам 

#### Недельный анализ по часам

In [ ]:
dh = seasonal_decompose(df['2018-07-02':'2018-07-08'])

dh.seasonal.plot(figsize=(20,8),lw=2,title='Сезонная составляющая и остаток за первую неделю июля' );
dh.resid.plot(figsize=(20,8),ls='--');
plt.legend();

По часам также есть сильные колебания, нельза выделить какой-ту особенную часть суток, количество заказов сильно меняются от часа к часу, со значительными нерегулярными остатками.  
Пики заказов на часы пик и полночь, минимумы - ночью

In [ ]:
dh_a = seasonal_decompose(df['2018-07-30':'2018-08-06'])
dh_a.seasonal.plot(figsize=(20,8),lw=2,title='Сезонная составляющая и остаток за первую неделю августа ');
dh_a.resid.plot(figsize=(20,8),ls='--');
plt.legend();

По сравнению с июльским графиком есть небольшие  изменения -более плоский спад после получночи и плоский максимум в часы пик .   
Но понять , в какое время суток идёт увеличение заказов в понедльники, мне не удалось 

#### Разностный анализ

In [ ]:
def plot_diff(dx,sf,title):
    dxc = seasonal_decompose( (dx-dx.shift(sf)).dropna() )
    dxc.seasonal.plot(figsize=(20,8),lw=2,title=title);
    dxc.resid.plot(figsize=(20,8),ls='--');
plot_diff(df['2018-07-02':'2018-07-16'],1, title='Почасовая разница в заказых за 2-15 июля ')                  

Почесовые разницы так же нестабильны, как и число заказов

In [ ]:
plot_diff(df['2018-07-02':'2018-07-16'],24, title='Поcуточная  разница в количестве заказов за час за 2-15 июля ')

Суточные разности выглядят намного более  стационарным процессом, чем исходное количество заказов 

In [ ]:
plot_diff(df['2018-07-02':'2018-07-30'],24*7, title='Понедельная  разница в количестве заказов за час за 2-30 июля ')

Также как и недельные разницы.   
**Основной вывод по резултатам анализа -  
 существование суточных  и недельных циклов количества заказов**

## Обучение

Так как есть выраженные периодические  колебания по дням недели и в течение суток, я ожидаю, что будет достаточно двуз признаков - час и день недели (категориальные) , а также для учета тренда - количество дней с начала года (числовой признак).
Но на всякий случай я добавлю сдвиги на дни и часы а также учет среднего за две недели.
В тестировании будут проверены оба варианта- с дополнительными признаками и без

#### Подготовка признаков

In [ ]:
def split_date(d ):                    
    return d,d.dayofyear,d.day_name(),d.hour 

df = df.join( df.index.map(split_date).to_frame(name=['datetime','doy','day','hour']).set_index('datetime') )


In [ ]:
for i in range(1,4):
    df["shift_"+str(i)] = df.num_orders.shift(i)
for i in [1,2,7]:
    df["shift_"+str(i*24)] = df.num_orders.shift(i*24)    
for i in [15]:     
    df['avg'+str(i)] = df.num_orders.shift().rolling(i).mean() 
df = df.dropna()    
df.info()
df.head()    

In [ ]:
df[['num_orders']].join( df.num_orders.shift().rolling(15).agg(['mean']) ).dropna().head(10)

In [ ]:
cat_cols = ['day','hour']
df[cat_cols]= df[cat_cols].astype('category')
# num_cols0 = ['doy']
num_cols = [ c for c in df.columns if df.dtypes[c] in ['int64','float64'] ]
print( cat_cols,num_cols)


#### Разбиение на тестовый и тренировочный набор 

При этом поддерживаются наборы Xs c сокращенным количеством признаков (без сконструированных со сдвигом и средним) 

In [ ]:
y = 'num_orders'
X = [ c for c in num_cols + cat_cols if c != y ] 
Xs =  cat_cols +[ 'doy' ]


X_tr,X_te,y_tr,y_te= train_test_split(df[X],df[y], test_size=.1,shuffle= False)
Xs_tr = X_tr[Xs]
Xs_te = X_te[Xs]
print(X_tr.index.max(),X_te.index.min())


In [ ]:
X_tr.head()

## Тестирование

Я использовал библиотеку классов из прошлых заданий с небольшими доработками. 
Она включает класс для разбора результата кроссвалидации, два класса рисования графиков и класс для интеграции теста  
ДЛя поиска оптимальных параметров на тренировочном наборе используется кроссвалидатор с разбиением временных рядов TimeTestSplit

####  ParsedResult

Класс разбора результата кроссвалидации, формирует 

In [ ]:
class ParsedResult:
    def __init__(self,result,metric_name='score',sample_name='mean'):
        self.df = pd.DataFrame.from_dict( 
            { k:v for k,v in result.items() if k not in ['params']}
        )
# drop param_ prefix from column names for tidy display         
        self.primary_score=f"{sample_name}_test_{metric_name}"
        param_names = [ c for c in self.df.columns if c[:6]=='param_' ]        
        self.param_cols = [ c[6:] for c in param_names ]              
        self.df = self.df.rename( columns={ p:c for p,c in zip(param_names,self.param_cols) } )
# also drop model_ prefix which we use for model grid in pipeline   
        model_names = [ c for c in self.df.columns if c[:7]=='model__' ]
        self.param_cols = [c[7:]  if c[:7]=='model__' else c for c in self.param_cols ]
        self.model_cols = [ c[7:] for c in model_names ]   
        self.df = self.df.rename( columns={ p:c for p,c in zip(model_names,self.model_cols) } )
# select parameters, for which tested several values and arrange them descending (by number of differnt values)
        values_per_cols = list(zip( self.param_cols,[ len( np.unique( self.df[c].values ) ) for c in self.param_cols]))
        self.multi_value_cols =[it[0] for it in sorted(values_per_cols, key=lambda pair:pair[1],reverse=True) if it[1]>1 ]
# some object columns may be not string (f.e. lists)
        for i in range(len(self.df.columns)):
            if self.df.dtypes[i]=='object':
                self.df[self.df.columns[i]] = self.df[self.df.columns[i]].astype(str)
        
            
    def select(self, index,values=[],filters={},agg=[] ):
        values = [ [self.primary_score] ,values][bool(values)]
        if len(filters)>0 :
            condition = 'and'.join([ f" {k}=={v} " for k,v in filters.items() ]) 
            filtered = self.df.query(condition)    
        else:
            filtered = self.df
        cols = [ c for c in self.multi_value_cols if c not in list(filters.keys())+[index]+values ]      
        return(self.df.pivot_table(index=index,values=values,columns=cols)  )
     

#### ResultPlotter

Два класса для однородной отрисовки графиков :
Вспомогательный класс формирования аттрибутов линий для графиков, создаваемых в основном плоттере


In [ ]:
class Styler:
    base_array=[]
    def __init__(self,param_array=[]):
        self.params = [ self.base_array,param_array][bool(param_array)]
    
    def key(self):
        pass
    
    def put(self,dct,index):
        dct[self.key()] = self.params[index]
        
class ColorStyler(Styler):
    base_array=[*'rgbykm']
    def key(self):
        return('color')

class DashStyler(Styler):
    base_array=['--',':','-.','-']
    def key(self):
        return('ls')
    
class WidthStyler(Styler):
    base_array=[ 2,4,7]
    def key(self):
        return('lw')

class AlphasStyler(Styler):
    base_array=[ .3,.9]
    def key(self):
        return('alpha')
    

И собственно плоттер, рисующий каждый столбец мультиколонки в своем стиле.   
Отельные непересекающиеся аттрибуты создаются для каждого уровня мультиколонки,   
в результате каждая элементарная подколонка получает уникальный стиль   

In [ ]:
class ResultPlotter:
    base_params = {
        'colors':[*'rgbykm'],
        'styles':['--',':','-.','-',(0, (1, 10)),(0, (5, 10))],
        'widths':[ 2,4,7],
        'alphas':[.3,.8],
        'figsize': (20,8),
        'logscale':""
    }
    
    def __init__(self, test, params = {} ):
        self.model = test.model
        self.df= test.sel
        self.params=self.base_params | params
        self.stylers = [ColorStyler(self.params['colors']),DashStyler(self.params['styles']),
                        WidthStyler(self.params['widths']),AlphasStyler(self.params['alphas'])]
        self.mcols = self.df.columns      
        self.lev_names = list(self.mcols.names)
        self.num_levels= len(self.lev_names)
        self.level_sizes=[ len(set(self.mcols.get_level_values(x))) for x in self.lev_names]
        
        # if level 0("values of pivot") is one-valued, then start apply styles from columns 1  
        self.start_level = int(  self.level_sizes[0] == 1   )
# TODO: refactoring needed here, one or two new functions to encapsulate this feature
#       also to print short description for " result thread" 

        

# auxliary func to forms sequence of indexes for levels
# result is need index on the level  which may be one of color|dashstyle|linewidth
    def idx_on_level(self,idx, lev_num):
        items_on_hyperplane = multiply(self.level_sizes[(lev_num+1):]) 
        return idx//items_on_hyperplane% self.level_sizes[lev_num]
    
    def plot(self):
        fig,ax = plt.subplots(figsize=(20,6))
        if 'x' in self.params['logscale'].lower():
            ax.set_xscale('log')
        if not self.lev_names[0]:
            self.lev_names[0]='res.thread'
            
        for i_mcol in range(len( self.mcols)):
            graph_params = {}
            for i_level in range(self.start_level,self.num_levels):
                self.stylers[i_level-self.start_level].put( graph_params, self.idx_on_level(i_mcol,i_level) ) 
            idx=self.mcols[i_mcol]
            
            levels_for_label=[ f"{n}:{j} " for n,j in zip(self.lev_names[self.start_level:],idx[self.start_level:]) ] 
            graph_params['label'] = ','.join( levels_for_label )
            ax.plot(self.df[idx] ,**graph_params)
            ax.set_title(f"Тест {self.params['title']}")
            ax.set_xlabel('learning rate')
            ax.set_ylabel(f"{self.params['metric']}")
            ax.legend()

#### Класс для универсального теста на модели

Содержит программу стандартного теста

In [ ]:
class TestModel:
    base_params = {
        'cv':4 ,
        'scoring' : ['neg_root_mean_squared_error','r2']
    }
    def __init__(self, model, params = {} ):
        self.model = model
        self.params = self.base_params | {'title':type(self.model).__name__} | params 
        
    def  __getattr__(self, k):
        if k in self.params:
            return ( self.params[k] )  
        
    def cross_validate(self,param_grid,X_tr,y_tr,refit=None):
        self.refit = [self.scoring[0],refit ][bool(refit)]
        self.gcv = GridSearchCV(self.model, param_grid,cv= self.cv, scoring=self.scoring,refit=self.refit) 
        self.gcv.fit(X_tr,y_tr)
        self.best = self.gcv.best_params_
        return self.best
    
    def check_best(self,X_tr,y_tr,X_te,y_te,metrics=[rmse,r2_score]):
        model = self.model.set_params(**self.best)
        model.fit(X_tr,y_tr)
        pr = model.predict(X_te)
        for metric in metrics:
            print(f"{metric.__name__}: {metric(y_te,pr)} \t")

    
    def parse_result(self,index_column='learning_rate',values=[]):
        self.res = ParsedResult(self.gcv.cv_results_,metric_name=self.refit)
        self.sel = self.res.select(index_column,values=values)
        return self.sel
        
    def plot(self):
        self.rp = ResultPlotter(self,{'logscale':'x','title':self.title,'metric':self.refit})
        self.rp.plot()

In [ ]:
%%time
tscv = TimeSeriesSplit(10)
#tscv5 = TimeSeriesSplit(10,max_train_size =5)
                
debug_grid = {
    'iterations': [64,128],
    'learning_rate': [.1,.03,.01,.003]
}

test_grid = {
    'iterations': [64,128,256,512,1024],
    'learning_rate': [.6,.3,.1,.06,.03,.02,.01,.003]
}
param_grid = test_grid

#### Тест Catboost на полном наборе признаков  

In [ ]:
%%time
cbst = TestModel(CatBoostRegressor(logging_level='Silent',cat_features=cat_cols),
              params=   {'cv': tscv,'title':'CatBoost full features set'} )
cbst.cross_validate(param_grid,X_tr,y_tr)

Этот метод проверяет лучший результат на тестовом наборе 

In [ ]:
cbst.check_best(X_tr,y_tr,X_te,y_te)

 на тестовом наборе достигнута <b> метрика RMSE 38.7  </b>

 В принципе, этого  достаточно, так как требуется 48   
 Посмотрим что получается при различных вариантов гиперпараметров

In [ ]:
cbst.parse_result()

In [ ]:
cbst.sel.columns

In [ ]:
cbst.plot()

Максимумы достигаются при скорости 0.02-.01 в зависимости от числа оценщиков   
Результаты вглядят стабильно без резких колебаний  а приизменение и скоростей в ту или другую сторону начинается пере- или недо- обучение  
Однако заметно серьезное отличие средней  метрики тестов(mean_test_score) от  результата финального теста    

Поэтому отдельно выберем результаты последних серий и посмотрим метрику для них

In [ ]:
cbst.parse_result(values=['mean_test_neg_root_mean_squared_error',
                          'split6_test_neg_root_mean_squared_error',
                          'split8_test_neg_root_mean_squared_error',
                          'split9_test_neg_root_mean_squared_error'])

In [ ]:
cbst.plot()

К сожалению, финальные серии предсказывающие крайние результаты (желтые , split_9) дают гораздо худший результат , чем ранние (split_6). Я это связываю с тем, что характер предсказываемой кривой в августе несколько изменился.

Характер кривых скоринга  все же не меняется и максимумы так же на .03-.1  с максимальным числом оценщиков,
 так что выбор лучшей модели не меняется .
 
Возникает вопрос -  как лучше отбирать тренировочную выборку для подбора гиперпараметров ? 
Возможно нужно "расширять" тренировочное окно назад и сравнивать результаты, но простого сопсоба сделать это я не нашел. 

#### Тест на сокращенном наборе признаков 

In [ ]:
%%time
cbsts = TestModel(CatBoostRegressor(logging_level='Silent',cat_features=cat_cols),
                  params=   {'cv': tscv,'title':'CatBoost short features set'} )
cbsts.cross_validate(param_grid,Xs_tr,y_tr)

In [ ]:
cbsts.check_best(Xs_tr,y_tr,Xs_te,y_te)

In [ ]:
cbsts.parse_result()

In [ ]:
cbsts.plot()

Выбор оптимальной модели тот же, но результаты несколько хуже.  
Так что дополнительные признаки все же полезны

#### LGBM Test

In [ ]:
debug_grid = {
    'iterations': [64,128],
    'learning_rate': [.1,.03,.01,.003]
}

test_grid = {
    'boosting': ['gbdt','dart','goss'],
    'iterations': [64],
    'learning_rate': [.6,.3,.1,.03,.01,.003]
}
param_grid = test_grid

In [ ]:
%%time
lgbm = TestModel(LGBMRegressor(verbose=-1),
                  params=   {'cv': tscv,'title':'LGBM'} )
lgbm.cross_validate(param_grid,X_tr,y_tr)

In [ ]:
lgbm.check_best(X_tr,y_tr,X_te,y_te)

In [ ]:
lgbm.parse_result()

In [ ]:
lgbm.plot()

Модель LGBM показала несколько худший результат , чем CatBoost.  
Оптимальные скорости обучения так же около .03-.1, хотя на неоптимальных падение качества обучения идет быстрее , чем у CatBoost  
В качестве оптимальной модели выбрана лучшая модель Catboost

### ВЫВОДЫ

* Количество заказов монотонно растет по неделям , общий рост за полгода - примерно 20% 
* В количестве заказов наблюдаются недельные и суточные колебания
* Максимальное количество заказов в понедельник и пятницу , минимальное - во вторник и воскресенье
* В течение суток максимумы заказов в полночь и в часы пик , минимумы - под утро
* Моделирование позволят предсказывать количество закзов в час со среднеквадратичным отклонением 40
* Лучшая предказывающая модель - Catboost, при  предсказании нужно учитывать час и день недели, важно также колчиество заказов неделю назад и среднее за две недели  